In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder

In [7]:
# from data.py
def get_data (csv_file):
    df = pd.read_csv('../raw_data/' + f'{csv_file}')
    return df

# NEW VERSION

In [684]:
#from matrix.py ######################### NEW VERSION ###########################################
### PENSER A UTILISER LA NOTION DE SAISON ??? ou pas ?
###################################################################################################

def encode_categories_articles(clusters_articles_df):  #df doit avoir les champs format ['article_id','catID']
    # création de dummies pour chaque catégorie de produits
    cluster_name=clusters_articles_df.columns[1]
    ohe = OneHotEncoder(sparse=False)# Instanciate encoder
    cat_encoded=pd.DataFrame(ohe.fit_transform(clusters_articles_df[[cluster_name]]))
    cat_names=ohe.categories_
    return pd.concat([articles_hetm[['article_id']],cat_encoded], axis=1) 


def build_segment_base(segment): # ce df contient toutes les transactions des individus du segment 
    return segment[['customer_id']].merge(transact_history)

def build_segment_horizon_transactions(segment_base,X_week,horizon):
    #filtrage sur l'horizon temporel pour connaitre les préférences des customers
    week_min= X_week-horizon+1
    week_max= X_week
    filtre=np.logical_and(segment_base['t_dat'] >= week_min,segment_base['t_dat'] <= week_max)
    Xh_week_achats=segment_base[filtre][['customer_id','t_dat','article_id']]
    Xh_week_achats['counter']=1
    return Xh_week_achats

def build_matrix(Xh_week_achats):
    achats_for_matrix=Xh_week_achats.drop(columns='t_dat')
    #achats_for_matrix['counter']=1 je pense pouvoir supprimer
    achats_for_matrix=achats_for_matrix.merge(articles_categories)

    # je fais un groupby par clients, sans le sort car j'ai ensuite besoin de refaire un tri
    # donc gain de temps de calcul car on suppose que sort_values est plus rapide que le sort de groupby
    achats_gb_cust_cat=achats_for_matrix.groupby(['customer_id'],as_index=False,sort=False)
    achats_gb_cust_cat_srt=achats_gb_cust_cat.sum().sort_values(by=['customer_id','counter'],ascending=False)

    # je refais ensuite un groupby par client pour calculer la somme de chaque dummy de categories
    segment_matrix=achats_gb_cust_cat_srt.drop(columns=['article_id']).groupby(['customer_id'],as_index=False).sum()
    return segment_matrix

def get_max_cats(Xh_week_achats,clusters_articles_df):
    ######################################################################################################
    #  catégories de produits achetés par customer, de la plus achetée (rank=1) à la moins achetée       #
    ######################################################################################################
    # contrairement à la construction de la matrice, ici je veut les infos de transactions en lignes
    tmp=Xh_week_achats.drop(columns='t_dat').merge(clusters_articles_df)
    cluster_name=clusters_articles_df.columns[1]
    cust_max_cats=tmp.groupby(['customer_id',cluster_name],as_index=False).sum()
    cust_max_cats.sort_values(by=['customer_id','counter'],ascending=False,inplace=True)
    cust_max_cats['rank']=cust_max_cats.groupby('customer_id')['counter'].rank(method='first',ascending=False)
    cust_max_cats['cat_perc']=cust_max_cats.groupby('customer_id', as_index=True, group_keys=False)['counter'].apply(lambda x: x / x.sum())
    return cust_max_cats

def calc_diversity_per_cust(cust_max_cats,threshold): #threshold est un pourcentage , exemple 30 pour 30%
    max_nb_cats=int(100/threshold)
    cust_max_cats['keep_cat']=cust_max_cats['cat_perc'].map(lambda x : 1 if x>=(threshold/100) else 0)
    # dans la ligne ci-dessous je suis obligée de lui mettre x/x (=1) dans la lambda fonction sinon il retourne NaN si dans le 
    # calcul on utilise pas la valeur x elle-même
    cust_max_cats['cust_diversity']=cust_max_cats.groupby('customer_id', as_index=True, group_keys=False)['keep_cat'].apply(lambda x: x/x * x.sum())
    return cust_max_cats

def top12_per_cat(segment_base,X_week,p,clusters_articles_df):
    # p est le nombre de périodes utilisées pour déterminer le top12 (différent de horizon)
    week_min= X_week-p+1
    week_max= X_week
    filtre=np.logical_and(segment_base['t_dat'] >= week_min,segment_base['t_dat'] <= week_max)
    tmp=segment_base[filtre].drop(columns=['customer_id','t_dat']).merge(clusters_articles_df)
    cluster_name=clusters_articles_df.columns[1]
    tmp['counter']=1
    tmp=tmp.groupby([cluster_name,'article_id'],as_index=False).sum()
    tmp.sort_values(by=[cluster_name,'counter'],ascending=False,inplace=True)
    tmp['rank']=tmp.groupby(cluster_name)['counter'].rank(method='first',ascending=False)
    keep_first_12=tmp['rank']<=12
    top12_cat=tmp[keep_first_12] 
    return top12_cat
    
def top_df_to_dict(top_df): # transforme le Top format dataframe à format dictionnaire
    #########j'ai un warning pour cette ligne de code ci après:
    #A value is trying to be set on a copy of a slice from a DataFrame.
    #Try using .loc[row_indexer,col_indexer] = value instead
    #############################################################################
    #top_df['article_id']=top_df['article_id'].astype(str) #nécessaire pour utiliser la méthode .join qui travaille sur des str
    cat_list=top_df.iloc[:,0].values
    dict_top_cats={}
    for cat in cat_list:
        mask=top_df.iloc[:,0]==cat
        tmp=top_df[mask]
        #je passe à ma clef de dictionnaire (= cat) une string contenant les articles_id associés à la cat
        articles_arr=tmp['article_id'].values
        #articles_str=" ".join(tmp['article_id'].values) 
        dict_top_cats[cat]= articles_arr #articles_str
    return dict_top_cats

############ je généralise la méthode ci-dessus à tout df qu'on veut transformer en dictionnaire##############
## group_name = nom de la série quon veut considérer comme un group_by 
## value_to_list = nom de la variable dont les valeurs prises par le groupe sont à passer dans une liste

def df_to_dict(df,group_name,value_to_list): # transforme le format dataframe à format dictionnaire
    df[value_to_list]=df[value_to_list].astype(str) #nécessaire pour utiliser la méthode .join qui travaille sur des str
    group_list=df.loc[:,group_name].values
    dict_group_values={}
    for group in group_list:
        mask=df.loc[:,group_name]==group
        tmp=df[mask]
        #je passe à ma clef de dictionnaire (=group) une string contenant les values associées au sein du groupe
        values_list=tmp[value_to_list].values
        #values_str=" ".join(tmp[value_to_list].values) 
        dict_group_values[group]=values_list
    return dict_group_values 
#########################################################################################################

def top12_filtre(top12_cat): # dans cette version l'algo gère nb_cat_to_predict in [1,2,3,4,6,12]
        nb_cat_to_predict=[1,2,3,4,6,12]
        for nb in nb_cat_to_predict:
            top_nb=12/nb
            if nb==1:
                top12x1cat=top12_cat
                dict_top12x1cat=top_df_to_dict(top12x1cat)
                top12x1cat.to_csv('top12x1cat.csv')
            elif nb==2:
                top6x2cat=top12_cat[top12_cat['rank']<=top_nb]
                dict_top6x2cat=top_df_to_dict(top6x2cat)
                top6x2cat.to_csv('top6x2cat.csv')
            elif nb==3:
                top4x3cat=top12_cat[top12_cat['rank']<=top_nb]
                dict_top4x3cat=top_df_to_dict(top4x3cat)
                top4x3cat.to_csv('top4x3cat.csv')
            elif nb==4:
                top3x4cat=top12_cat[top12_cat['rank']<=top_nb]
                dict_top3x4cat=top_df_to_dict(top3x4cat)
                top3x4cat.to_csv('top3x4cat.csv')
            elif nb==6:
                top2x6cat=top12_cat[top12_cat['rank']<=top_nb]
                dict_top2x6cat=top_df_to_dict(top2x6cat)
                top2x6cat.to_csv('top2x6cat.csv')
            elif nb==12:    
                top1x12cat=top12_cat[top12_cat['rank']<=top_nb]
                dict_top1x12cat=top_df_to_dict(top1x12cat)
                top1x12cat.to_csv('top1x12cat.csv')
        return dict_top12x1cat#,dict_top6x2cat,dict_top4x3cat,dict_top3x4cat,dict_top2x6cat,dict_top1x12cat
    

def baseline0(segment,X_week,q): #supprimer tdat et restituer les valeurs plutot que df
    # q est le nbre de weeks sur lesquels on veut définir les articles_id de la baseleine
    segment_base= build_segment_base(segment)
    Xq_week_achats=build_segment_horizon_transactions(segment_base,X_week,q)
    Xq_week_achats.drop(columns='t_dat', inplace=True)
    Xq_week_achats['counter']=1
    segment_top12=Xq_week_achats.groupby(['article_id'],as_index=False).sum().sort_values(by='counter', ascending=False).head(12)
    segment_baseline=segment_top12['article_id'].values
    return  segment_baseline
    
def fit_recommender(segment_df,clusters_articles_df,X_week,horizon,p,cat_threshold):
  
    # q est le nbre de weeks sur lesquels on veut définir les articles_id de la baseline
    # à chaque customer de mon segment je vais associer sa prediction (liste de 12 articles)
    # 1.construction des infos clients
    cluster_name=clusters_articles_df.columns[1]
    segment_base=build_segment_base(segment_df)
    Xh_week_achats=build_segment_horizon_transactions(segment_base,X_week,horizon)
    cust_max_cats=get_max_cats(Xh_week_achats,clusters_articles_df)
    cust_max_cats=calc_diversity_per_cust(cust_max_cats,cat_threshold)
    filtre=cust_max_cats['keep_cat']==1
    cust_max_cats_filtered=cust_max_cats[filtre]
    dict_cust_cat=df_to_dict(cust_max_cats_filtered,'customer_id',cluster_name)
    
    # 2.construction des infos articles à faire correspondre ensuite aux clients
    top12_cat=top12_per_cat(segment_base,X_week,p,clusters_articles_df)
    #old version : top_df=top12_split(top12_df,diversity)
    #old version: dict_cat_articles=top_df_to_dict(top_df)
    #old version:dict_cat_articles=top12_filtre(top12_cat)
    dict_cat_articles=top_df_to_dict(top12_cat)
    return dict_cust_cat, dict_cat_articles
    
def build_pred_base(dict_cust_cat, dict_cat_articles, segment_baseline):
    # paires (diversité client : nb de liste articlesID à chercher)
 
    # buyers_default_strategy : si les clients avec achats sur l'horizon choisi n'ont pas le nbre de catégories correspondant
    # au niveau de diversité désiré
    # cold_start_default_strategy : quelle baseline donner aux clients sas acahts sur l'horizon choisi 
    # (Baseline0 calculée sur quels mois)
    #dict_cust_cat, dict_cat_articles=fit_recommender(segment_df,clusters_articles_df,X_week,horizon,p,diversity)
    
    # 1. customers avec achats dans l'horizon choisi
    dict_cust_articles={}
    for customer,cat_list in dict_cust_cat.items():
        nb_cat_to_predict=min(len(cat_list),12)
        nb_articles_per_cat=int(12/nb_cat_to_predict)
        modulo=12 % nb_cat_to_predict
        articles_arr=[[]]
        if nb_cat_to_predict==0:
            articles_arr= segment_baseline
        elif modulo==0 and nb_cat_to_predict==1:
            cat=cat_list[0]
            articles_arr=np.r_[dict_cat_articles[cat][:6],segment_baseline[:6]]# on force la diveristé en mettant du baseline0
        else: # modulo positif ou nul
            from_baseline=0
            for cat in cat_list:
                nb_articles_in_cat=len(dict_cat_articles[cat])
                if nb_articles_in_cat<nb_articles_per_cat:
                    start_index=from_baseline
                    end_index= start_index+nb_articles_per_cat-nb_articles_in_cat                   
                    articles_arr=np.r_[dict_cat_articles[cat][:nb_articles_in_cat],segment_baseline[start_index:end_index]]
                    from_baseline=from_baseline+(nb_articles_per_cat-nb_articles_in_cat)
                else :
                    articles_arr=np.r_[articles_arr,dict_cat_articles[cat][:nb_articles_per_cat]]
        if modulo>0 :# modulo positif donc à compléter avec du baseline
            start_index=from_baseline
            end_index= start_index+modulo
            articles_arr=np.r_[articles_arr,segment_baseline[start_index:end_index]]
        dict_cust_articles[customer]=articles_arr
    
    return articles_arr# dict_cust_articles

def y_true(segment_base,y_week,t):
    #segment_base= build_segment_base(segment)
    y_week_achats=build_segment_horizon_transactions(segment_base,y_week,t)
    y_week_achats.drop(columns='t_dat', inplace=True)
    y_week_achats['counter']=1
    segment_ytrue_df=y_week_achats.groupby(['customer_id','article_id'],as_index=False).sum().sort_values(by='counter', ascending=False)
    segment_ytrue_dict=df_to_dict(segment_ytrue_df,'customer_id','article_id')
    return segment_ytrue_dict
    
def pred(customer): 
    # chercher si customer(s) est/sont dans dict_cust_articles et si oui lui/leur donner la liste d'articles correspondante
    # sinon attribuer la baseline
    
    pass

In [682]:
def build_pred_base(dict_cust_cat, dict_cat_articles, segment_baseline):
    # paires (diversité client : nb de liste articlesID à chercher)
 
    # buyers_default_strategy : si les clients avec achats sur l'horizon choisi n'ont pas le nbre de catégories correspondant
    # au niveau de diversité désiré
    # cold_start_default_strategy : quelle baseline donner aux clients sas acahts sur l'horizon choisi 
    # (Baseline0 calculée sur quels mois)
    #dict_cust_cat, dict_cat_articles=fit_recommender(segment_df,clusters_articles_df,X_week,horizon,p,diversity)
    
    
    dict_cust_articles={}
    for customer,cat_list in dict_cust_cat.items():
        nb_cat_to_predict=min(len(cat_list),12)
        nb_articles_per_cat=int(12/nb_cat_to_predict)
        modulo=12 % nb_cat_to_predict
        articles_arr=[[]]
        articles_tmp=[[]]
        if nb_cat_to_predict==0:
            articles_arr= segment_baseline
        elif modulo==0 and nb_cat_to_predict==1:
            cat=cat_list[0]
            articles_arr=np.r_[dict_cat_articles[cat][:6],segment_baseline[:6]]
            print(articles_arr)
        else: # modulo positif ou nul
            from_baseline=0
            for cat in cat_list:
                nb_articles_in_cat=len(dict_cat_articles[cat])
                articles_tmp=np.r_[articles_tmp,dict_cat_articles[cat][:nb_articles_in_cat]]
            diff_size=12- len(articles_tmp)
            print(size)
    return customer

In [653]:
dict_cust_cat

{'fede72be6d8a2130624098f344c15f4d88a45d8b3cbbc9f21e1964a471b394b2': array(['Accessories'], dtype=object),
 'feb01a2d17ca601bb4361099013b9010773778a3da35524742e998c530920c8d': array(['Trousers'], dtype=object),
 'fe452b72e97790e68dcf79f49521782d486c345981f773a16160db6fa2b28567': array(['Jersey Basic'], dtype=object),
 'fddef481e1e792752bb5b7243c1345cb0379a74c64722d7a321a9cdae7d893a0': array(['Trousers'], dtype=object),
 'fd458cc672b5dbd88d7cd1e92bb289f1a5f20d12bcda610616fea92f725d3320': array(['Jersey Fancy', 'Swimwear'], dtype=object),
 'fd0d0860c52aea5a8b38b01550603e88fe5a6008e19eca98308a57523272a587': array(['Under-, Nightwear'], dtype=object),
 'fc90e1611724a9ca812fe2dd8038adc312b49dab92a4b254b02cab13605292e2': array(['Jersey Basic', 'Knitwear'], dtype=object),
 'fc56c2b297803575c07f819af0dc8fd73c68eb7ad13454468bb83b77a5ef363f': array(['Jersey Basic', 'Knitwear'], dtype=object),
 'fbf0c743801ed29a62f46b8d562edae6d7eabe6da8f5a90c237b53a43235ef4f': array(['Jersey Basic'], dtype=objec

In [680]:
#dict_cust_cat_test={'fede72be6d8a2130624098f344c15f4d88a45d8b3cbbc9f21e1964a471b394b2': ['Accessories']}
dict_cust_cat_test={'fadc6e3eac34bc261ee8a8b155769263891462f9551d1b45786419e7f7546c97': ['Accessories', 'Knitwear', 'Under-, Nightwear']}

In [683]:
build_pred_base(dict_cust_cat_test,dict_cat_articles,S1_baseline0)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [11]:
# Récupértaion de segments clients de Naceur selon l'âge
# note du 06042022: à ce stade de l'analyse les customers sans information sur l'âge ne sont pas inclus
#get_data('customers_age_16_17.csv')
#get_data('customers_age_18_19.csv','S2')
#get_data('customers_age_20_21.csv','S3')
#get_data('customers_age_22_23.csv','S4')
#get_data('customers_age_24_25.csv','S5')
#get_data('customers_age_26_27.csv','S6')
#get_data('customers_age_28_29.csv','S7')
#get_data('customers_age_30_34.csv','S8')
#get_data('customers_age_35_39.csv','S9')
#get_data('customers_age_40_44.csv','S10')
#set_data('customers_age_45_49.csv','S11')
#get_data('customers_age_50_54.csv','S12')
#get_data('customers_age_55_59.csv','S13')
#get_data('customers_age_60.csv','S14')
S1=get_data('customers_age_16_17.csv')

In [12]:
# Récupération du petit fichier de transactions "test" contenant semaines 101 à 104:
transactions_file='transaction_train_week_101_104.csv'
transactions_file_path=f'../raw_data/{transactions_file}'
transact_history=get_data(transactions_file_path)

In [13]:
#cleaning du petit fichier utilisé pour tester les programmes
transactions_101_104 = transact_history
transactions_101_104.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'price', 'sales_channel_id'], inplace=True)

In [14]:
#construction de la base_segment utile pour le calcul des valeurs des éléments de la matrice
S1_base=build_segment_base(S1)

In [15]:
S1_base

,customer_id,t_dat,article_id
0,0006ca2c5ecdbdcc1de5e72adffc6c33448fc2aaa62afe...,101,572998001
1,0006ca2c5ecdbdcc1de5e72adffc6c33448fc2aaa62afe...,101,572998009
2,00089f13f465ec902e5c49a3bb408c5e31205096d6f267...,101,926502001
3,000e3f587242eb077685a487ad27dad632a4801576dfd1...,101,501620049
4,001c42459c8978d55a69fa1528975893b011a6539631f3...,101,885077003
...,...,...,...
5950,ffd6bb6d6a33e415467e69d18d28ec9e64061a4f58d67c...,104,914441006
5951,ffd6bb6d6a33e415467e69d18d28ec9e64061a4f58d67c...,104,914441004
5952,ffec95723bd3cfc50cef76f198716d18e395261f7cd8b1...,104,858147006
5953,ffec95723bd3cfc50cef76f198716d18e395261f7cd8b1...,104,892643001


In [16]:
#création des clusters d'articles
articles_hetm = get_data('../raw_data/articles.csv')
articles_hetm=articles_hetm[['article_id','garment_group_name']] #fait office de clusters_articles_df
articles_categories=encode_categories_articles(articles_hetm)

In [17]:
articles_categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 22 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   article_id  105542 non-null  int64  
 1   0           105542 non-null  float64
 2   1           105542 non-null  float64
 3   2           105542 non-null  float64
 4   3           105542 non-null  float64
 5   4           105542 non-null  float64
 6   5           105542 non-null  float64
 7   6           105542 non-null  float64
 8   7           105542 non-null  float64
 9   8           105542 non-null  float64
 10  9           105542 non-null  float64
 11  10          105542 non-null  float64
 12  11          105542 non-null  float64
 13  12          105542 non-null  float64
 14  13          105542 non-null  float64
 15  14          105542 non-null  float64
 16  15          105542 non-null  float64
 17  16          105542 non-null  float64
 18  17          105542 non-null  float64
 19  18

In [18]:
articles_categories.head()

,article_id,0,1,2,3,4,5,6,7,8,...,11,12,13,14,15,16,17,18,19,20
0,108775015,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,108775044,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,108775051,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,110065001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,110065002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [19]:
S1_Xh_week_achats=build_segment_horizon_transactions(S1_base,103,1)

In [20]:
S1_Xh_week_achats.shape

(885, 4)

In [21]:
S1_Xh_week_achats

,customer_id,t_dat,article_id,counter
18,00953d607bb9b498cf020c314436b74c54876e1f03739a...,103,806388003,1
19,00953d607bb9b498cf020c314436b74c54876e1f03739a...,103,806388018,1
20,00953d607bb9b498cf020c314436b74c54876e1f03739a...,103,806388001,1
21,00953d607bb9b498cf020c314436b74c54876e1f03739a...,103,806388001,1
22,00953d607bb9b498cf020c314436b74c54876e1f03739a...,103,866731001,1
...,...,...,...,...
5866,fd458cc672b5dbd88d7cd1e92bb289f1a5f20d12bcda61...,103,853881001,1
5871,fddef481e1e792752bb5b7243c1345cb0379a74c64722d...,103,621381014,1
5899,fe452b72e97790e68dcf79f49521782d486c345981f773...,103,487800051,1
5907,feb01a2d17ca601bb4361099013b9010773778a3da3552...,103,706016003,1


In [22]:
build_matrix(S1_Xh_week_achats)

,customer_id,counter,0,1,2,3,4,5,6,7,...,11,12,13,14,15,16,17,18,19,20
0,00953d607bb9b498cf020c314436b74c54876e1f03739a...,5,0.0,0.0,0.0,0.0,0.0,4.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0105f638a3d98cd1918028de84d296e2ec5f075d5ccf3b...,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,017e589a1b5990b94288741a86252e14c1c361f3802827...,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,026e36e2a2131e0bdeb78752d03c3caad8932f0e361a3a...,1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0285d6fbe7c6db42c94a540de427c4f24c30e863d4ca18...,4,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,fd458cc672b5dbd88d7cd1e92bb289f1a5f20d12bcda61...,2,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
340,fddef481e1e792752bb5b7243c1345cb0379a74c64722d...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
341,fe452b72e97790e68dcf79f49521782d486c345981f773...,1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
342,feb01a2d17ca601bb4361099013b9010773778a3da3552...,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0


In [254]:
cust_max_cats=get_max_cats(S1_Xh_week_achats,articles_hetm)

In [255]:
cust_max_cats.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 602 entries, 601 to 1
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   customer_id         602 non-null    object 
 1   garment_group_name  602 non-null    object 
 2   article_id          602 non-null    int64  
 3   counter             602 non-null    int64  
 4   rank                602 non-null    float64
 5   cat_perc            602 non-null    float64
dtypes: float64(2), int64(2), object(2)
memory usage: 32.9+ KB


In [256]:
cust_max_cats

,customer_id,garment_group_name,article_id,counter,rank,cat_perc
601,fede72be6d8a2130624098f344c15f4d88a45d8b3cbbc9...,Accessories,896123001,1,1.0,1.0
600,feb01a2d17ca601bb4361099013b9010773778a3da3552...,Trousers,706016003,1,1.0,1.0
599,fe452b72e97790e68dcf79f49521782d486c345981f773...,Jersey Basic,487800051,1,1.0,1.0
598,fddef481e1e792752bb5b7243c1345cb0379a74c64722d...,Trousers,621381014,1,1.0,1.0
596,fd458cc672b5dbd88d7cd1e92bb289f1a5f20d12bcda61...,Jersey Fancy,853881001,1,1.0,0.5
...,...,...,...,...,...,...
4,026e36e2a2131e0bdeb78752d03c3caad8932f0e361a3a...,Dresses Ladies,832084003,1,1.0,1.0
3,017e589a1b5990b94288741a86252e14c1c361f3802827...,Accessories,639448001,1,1.0,1.0
2,0105f638a3d98cd1918028de84d296e2ec5f075d5ccf3b...,Jersey Basic,685813043,1,1.0,1.0
0,00953d607bb9b498cf020c314436b74c54876e1f03739a...,Jersey Basic,3225552023,4,1.0,0.8


In [220]:
cust_max_cats.iloc[596,:1].values

array(['0285d6fbe7c6db42c94a540de427c4f24c30e863d4ca18f0ca611d4938ab1a48'],
      dtype=object)

In [257]:
cust_max_cats[cust_max_cats['customer_id']=='0285d6fbe7c6db42c94a540de427c4f24c30e863d4ca18f0ca611d4938ab1a48']

,customer_id,garment_group_name,article_id,counter,rank,cat_perc
5,0285d6fbe7c6db42c94a540de427c4f24c30e863d4ca18...,Jersey Basic,1431248020,2,1.0,0.5
6,0285d6fbe7c6db42c94a540de427c4f24c30e863d4ca18...,Trousers,1791672006,2,2.0,0.5


In [258]:
cust_max_cats[cust_max_cats['customer_id']=='00953d607bb9b498cf020c314436b74c54876e1f03739a7e8312af21b89d83a5']

,customer_id,garment_group_name,article_id,counter,rank,cat_perc
0,00953d607bb9b498cf020c314436b74c54876e1f03739a...,Jersey Basic,3225552023,4,1.0,0.8
1,00953d607bb9b498cf020c314436b74c54876e1f03739a...,Jersey Fancy,866731001,1,2.0,0.2


In [380]:
cust_max_cats=calc_diversity_per_cust(cust_max_cats,30)

In [381]:
check=cust_max_cats[cust_max_cats['cust_diversity']==0]
check['customer_id'].values

array([], dtype=object)

In [382]:
cust_max_cats['cust_diversity'].value_counts()

1.0    229
2.0    164
3.0     69
Name: cust_diversity, dtype: int64

In [383]:
cust_max_cats['cat_perc'].value_counts()

1.000000    190
0.500000    130
0.333333     95
0.250000     66
0.200000     23
0.666667     23
0.166667     12
0.071429      7
0.750000      6
0.142857      5
0.125000      4
0.400000      4
0.285714      4
0.230769      3
0.222222      3
0.083333      3
0.714286      2
0.428571      2
0.076923      2
0.181818      2
0.100000      2
0.600000      2
0.090909      2
0.416667      2
0.300000      2
0.454545      1
0.153846      1
0.357143      1
0.555556      1
0.194444      1
0.800000      1
Name: cat_perc, dtype: int64

In [384]:
check=cust_max_cats[cust_max_cats['cust_diversity']==3]
check['customer_id'].values

array(['fadc6e3eac34bc261ee8a8b155769263891462f9551d1b45786419e7f7546c97',
       'fadc6e3eac34bc261ee8a8b155769263891462f9551d1b45786419e7f7546c97',
       'fadc6e3eac34bc261ee8a8b155769263891462f9551d1b45786419e7f7546c97',
       'f6d3408d9f8ceb660f7479fa574a642e0e19ecdd93cff0b446dbb7dad5f40cab',
       'f6d3408d9f8ceb660f7479fa574a642e0e19ecdd93cff0b446dbb7dad5f40cab',
       'f6d3408d9f8ceb660f7479fa574a642e0e19ecdd93cff0b446dbb7dad5f40cab',
       'de91cda47a2b95c123749c4ea4d1a9bd7048cb08908c1350fcc7d01deb01ce2d',
       'de91cda47a2b95c123749c4ea4d1a9bd7048cb08908c1350fcc7d01deb01ce2d',
       'de91cda47a2b95c123749c4ea4d1a9bd7048cb08908c1350fcc7d01deb01ce2d',
       'b6e9caf19f56ca9e6b8dbd0e843aeeef940c3db84176d617c1b678d4c4399d1a',
       'b6e9caf19f56ca9e6b8dbd0e843aeeef940c3db84176d617c1b678d4c4399d1a',
       'b6e9caf19f56ca9e6b8dbd0e843aeeef940c3db84176d617c1b678d4c4399d1a',
       '9c4a885009b1f75b84036898c5280185cb91b330b688ba97af98a5ae8ba73a99',
       '9c4a885009b1f75b8

In [385]:
cust_max_cats[cust_max_cats['customer_id']=='fadc6e3eac34bc261ee8a8b155769263891462f9551d1b45786419e7f7546c97']

,customer_id,garment_group_name,article_id,counter,rank,cat_perc,keep_cat,cust_diversity
580,fadc6e3eac34bc261ee8a8b155769263891462f9551d1b...,Accessories,916926005,1,1.0,0.333333,1,3.0
581,fadc6e3eac34bc261ee8a8b155769263891462f9551d1b...,Knitwear,874704002,1,2.0,0.333333,1,3.0
582,fadc6e3eac34bc261ee8a8b155769263891462f9551d1b...,"Under-, Nightwear",736923003,1,3.0,0.333333,1,3.0


In [386]:
cust_max_cats[cust_max_cats['customer_id']=='0285d6fbe7c6db42c94a540de427c4f24c30e863d4ca18f0ca611d4938ab1a48']

,customer_id,garment_group_name,article_id,counter,rank,cat_perc,keep_cat,cust_diversity
5,0285d6fbe7c6db42c94a540de427c4f24c30e863d4ca18...,Jersey Basic,1431248020,2,1.0,0.5,1,2.0
6,0285d6fbe7c6db42c94a540de427c4f24c30e863d4ca18...,Trousers,1791672006,2,2.0,0.5,1,2.0


In [331]:
top12_cat=top12_per_cat(S1_base,103,1,articles_hetm)

In [333]:
top12_cat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 166 entries, 594 to 3
Data columns (total 4 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   garment_group_name  166 non-null    object 
 1   article_id          166 non-null    int64  
 2   counter             166 non-null    int64  
 3   rank                166 non-null    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 6.5+ KB


In [334]:
top12_cat

,garment_group_name,article_id,counter,rank
594,Unknown,794321007,1,1.0
595,Unknown,815580001,1,2.0
596,Unknown,874937001,1,3.0
557,"Under-, Nightwear",748355003,4,1.0
558,"Under-, Nightwear",748355018,3,2.0
...,...,...,...,...
42,Accessories,910264001,2,8.0
0,Accessories,337991001,1,9.0
1,Accessories,516859013,1,10.0
2,Accessories,524465001,1,11.0


In [336]:
# vérification du rang sur certaines catégories
top12_cat['garment_group_name']=='Trousers'
top12_cat[top12_cat['garment_group_name']=='Trousers']

,garment_group_name,article_id,counter,rank
447,Trousers,621381012,12,1.0
448,Trousers,621381014,6,2.0
457,Trousers,706016002,6,3.0
433,Trousers,448509014,5,4.0
456,Trousers,706016001,5,5.0
438,Trousers,562245046,4,6.0
470,Trousers,760084003,4,7.0
487,Trousers,895836003,4,8.0
432,Trousers,448509001,3,9.0
441,Trousers,572998013,3,10.0


In [337]:
top12_cat['garment_group_name']=='Trousers Denim'
top12_cat[top12_cat['garment_group_name']=='Trousers Denim']

,garment_group_name,article_id,counter,rank
510,Trousers Denim,636323001,5,1.0
526,Trousers Denim,864216001,4,2.0
516,Trousers Denim,729860002,3,3.0
540,Trousers Denim,899749005,3,4.0
515,Trousers Denim,724906019,2,5.0
520,Trousers Denim,797835008,2,6.0
528,Trousers Denim,864224005,2,7.0
529,Trousers Denim,867966002,2,8.0
532,Trousers Denim,875272012,2,9.0
535,Trousers Denim,882899003,2,10.0


In [413]:
#pré calcule les top12 par degré de diversité possibles:
top12_filtre(top12_cat) # il faut les convertir en string

({'Unknown': array([794321007, 815580001, 874937001]),
  'Under-, Nightwear': array([748355003, 748355018, 923037001, 470789031, 668956005, 723469001,
         752689002, 892309003, 894756002, 918725001, 505882002, 506098007]),
  'Trousers Denim': array([636323001, 864216001, 729860002, 899749005, 724906019, 797835008,
         864224005, 867966002, 875272012, 882899003, 399256001, 399256023]),
  'Trousers': array([621381012, 621381014, 706016002, 448509014, 706016001, 562245046,
         760084003, 895836003, 448509001, 572998013, 573085042, 751471038]),
  'Swimwear': array([559633008, 685605001, 776237029, 832330003, 832331003]),
  'Special Offers': array([644288001, 909370001]),
  'Socks and Tights': array([889703001, 666444001, 932798001, 156231001, 182909001, 297067002,
         372860001, 372860002, 372860024, 783707046, 783707110, 783707123]),
  'Skirts': array([869379007, 777892001, 858856004, 869379001, 869379002, 869379011,
         904201001, 904201002, 918547001]),
  'Shoes

In [396]:
dict_cust_cat=fit_recommender(S1,articles_hetm,103,1,1,30)

/tmp/ipykernel_19098/2972836174.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[value_to_list]=df[value_to_list].astype(str) #nécessaire pour utiliser la méthode .join qui travaille sur des str


In [398]:
dict_cust_cat['fadc6e3eac34bc261ee8a8b155769263891462f9551d1b45786419e7f7546c97']

array(['Accessories', 'Knitwear', 'Under-, Nightwear'], dtype=object)

In [459]:
 dict_cust_cat, dict_cat_articles=fit_recommender(S1,articles_hetm,103,1,1,30)

/tmp/ipykernel_19098/458216695.py:98: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[value_to_list]=df[value_to_list].astype(str) #nécessaire pour utiliser la méthode .join qui travaille sur des str


In [460]:
dict_cust_cat['00953d607bb9b498cf020c314436b74c54876e1f03739a7e8312af21b89d83a5']    

array(['Jersey Basic'], dtype=object)

In [461]:
 dict_cust_cat['fadc6e3eac34bc261ee8a8b155769263891462f9551d1b45786419e7f7546c97']

array(['Accessories', 'Knitwear', 'Under-, Nightwear'], dtype=object)

In [462]:
 dict_cust_cat['0285d6fbe7c6db42c94a540de427c4f24c30e863d4ca18f0ca611d4938ab1a48']

array(['Jersey Basic', 'Trousers'], dtype=object)

In [463]:
dict_cat_articles

{'Unknown': array([794321007, 815580001, 874937001]),
 'Under-, Nightwear': array([748355003, 748355018, 923037001, 470789031, 668956005, 723469001,
        752689002, 892309003, 894756002, 918725001, 505882002, 506098007]),
 'Trousers Denim': array([636323001, 864216001, 729860002, 899749005, 724906019, 797835008,
        864224005, 867966002, 875272012, 882899003, 399256001, 399256023]),
 'Trousers': array([621381012, 621381014, 706016002, 448509014, 706016001, 562245046,
        760084003, 895836003, 448509001, 572998013, 573085042, 751471038]),
 'Swimwear': array([559633008, 685605001, 776237029, 832330003, 832331003]),
 'Special Offers': array([644288001, 909370001]),
 'Socks and Tights': array([889703001, 666444001, 932798001, 156231001, 182909001, 297067002,
        372860001, 372860002, 372860024, 783707046, 783707110, 783707123]),
 'Skirts': array([869379007, 777892001, 858856004, 869379001, 869379002, 869379011,
        904201001, 904201002, 918547001]),
 'Shoes': array([6192

In [424]:
S1_baseline0_df,S1_baseline0=baseline0(S1,103,1)

In [579]:
S1_baseline0

array([685814003, 621381012, 715624001, 685813043, 685814001, 682550002,
       621381014, 685814063, 706016002, 806388001, 706016001, 767423001])

In [676]:
predict(dict_cust_cat,dict_cat_articles,S1_baseline0)

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 1 has 1 dimension(s)

In [685]:
y_true(S1_base,104,1)

{'973953a3752f91bca95085d99bd530253cd0a21ecf555d96a167327546743069': array(['920610005', '913272003', '867966009', '751471001', '448509031',
        '871517001', '677930023', '448509018', '919037004', '448509014'],
       dtype=object),
 'fe452b72e97790e68dcf79f49521782d486c345981f773a16160db6fa2b28567': array(['831987001', '877389001', '880238002', '932642001', '467557008',
        '772902011', '865929003', '873217004', '757903029', '265630006',
        '816591005', '916468001', '456163085', '911214001', '826492003',
        '816919013', '890866001', '798891006', '772902001', '748355012'],
       dtype=object),
 '0f0d81edc152e71d6a8c269f1c11e3af11872e3eac9bafb30e9d49247127aa19': array(['918292011', '924605001', '885951002', '864415002'], dtype=object),
 '969aacd206c2da4d2e6d13a7cc678d7369e934a1b7eefa176e8b11a0eef3dd5b': array(['854818004', '732842021', '673396002', '877369001', '893429001',
        '933882001', '706016038', '706016002'], dtype=object),
 'dbb31b36adfd8cb3019f4c0f07e8d7